# Applied Machine Learning

This Jupyter Notebook presents our solution for the Applied ML Homework of the ADA course at EPFL.  

For this homework, we use a [football dataset](CrowdstormingDataJuly1st.csv) 
<sup id="fnref:1"><a href="#fn:1" rel="footnote">1</a></sup>  from a company for sports statistics, containing data from all soccer players (N = 2053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3147) that these players played under in their professional career.

The two main goals of this homework is to:
* Train a random forest classifier that given a soccer player description outputs his skin color
* Use an unsupervised learning technique to cluster the soccer players in 2 disjoint clusters, and try to find if there are relation between the clusters and the players skin color.

<div class="footnotes">
<hr/>
<ol>
<li id="fn:1">
<p>Since we all live in developped countries, we will use the term "football" instead of "soccer" for all the homework*<a href="#fnref:1" rev="footnote">&#8617;</a></p></li>
</ol>
</div>


In [2]:
import pandas as pd                                     
import numpy as np                                      
import os 



import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots


from os import path
from wordcloud import WordCloud
from PIL import Image
from gensim import corpora, models, similarities

/Users/jeremie/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [14]:
d = path.dirname("hillary-clinton-emails/")

# Read the whole text.
documents = open(path.join(d, 'Emails.csv')).read()


[From Strings to vector](https://radimrehurek.com/gensim/tut1.html)

In [22]:
stoplist = set('for a of the and to in - 1 2 3 4 5 6 7 8 9 a b c d e f g h i j k l m n o p q r s t u v w x y z é + " * ç % & / ()'.split())
print(stoplist)

{'7', 'n', 'ç', 'd', 'of', 'q', 'z', 'f', '8', 'k', '()', 'in', '2', '/', 'h', '6', 'w', 'y', 'c', 'i', 'x', 's', '&', '5', 't', '3', '*', 'é', 'the', 'r', 'p', 'v', '+', 'g', 'j', 'for', 'to', 'a', 'b', '%', '-', 'l', '"', '1', 'm', '9', 'u', 'o', 'and', 'e', '4'}


In [23]:
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

In [28]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
     for token in text:
            frequency[token] += 1

#texts = [[token for token in text if frequency[token] > 1]
#          for text in texts]

For representing the questions only by ids (intergers) we need a dictonary to store the mapping.

In [29]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester.dict")


In [32]:
print(dictionary)

Dictionary(56 unique tokens: ['–', '¶', '»', '÷', '(']...)


In [33]:
print(dictionary.token2id)

{'–': 35, '¶': 49, '»': 39, '÷': 37, '(': 8, '€': 43, '±': 36, '0': 1, '[': 14, '.': 4, '{': 34, 'ø': 31, '„': 26, '!': 21, '©': 19, '>': 7, '▪': 52, '_': 3, '›': 20, '<': 5, '✓': 50, '^': 16, '@': 6, '#': 32, '§': 44, '\\': 29, '=': 23, '➢': 48, '`': 30, ',': 0, '}': 27, '“': 53, 'ä': 47, '̀': 45, '”': 55, 'ã': 54, "'": 12, '�': 51, ';': 10, '~': 46, '£': 38, '‘': 28, '°': 17, '•': 15, '?': 13, ')': 9, '$': 18, ':': 2, '■': 33, '‹': 22, ']': 24, 'à': 40, 'ï': 42, '«': 25, '®': 41, '—': 11}


[Transformation interface](https://radimrehurek.com/gensim/tut2.html#available-transformations)